In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import category_encoders as ce # библиотека для кодирования признаков

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# 1. Загрузка данных и знакомство с ними

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info() # общая информация о таблице train

В данной таблице содержатся 386803 строк, 17 столбцов (8 столбцов формата 'object', 9 - числового формата ('float64','int64'). В столбцах 'lat' и 'lng' имеются пропуски. 

In [ ]:
df_train.head(2)

In [ ]:
df_test.info() # общая информация о таблице test

В данной таблице содержатся 128935 строк, 16 столбцов (8 столбцов формата 'object', 8 - числового формата ('float64','int64'). В столбцах 'lat' и 'lng' имеются пропуски.
нет столбца "reviewer_score", как в таблице train

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

# 2. Подготовка данных 

In [ ]:
display(data.describe())

При изучении статических данных числового формата было выявлено:
    есть аномальные значения в столбцах 'additional_number_of_scoring', 'total_number_of_reviews', 'total_number_of_reviews_reviewer_has_given', 'average_score', 'review_total_negative_word_counts', 'review_total_positive_word_counts'

In [ ]:
# Визуализируем столбцы с выбросами
fig, (ax1) = plt.subplots(nrows=1, ncols=6, figsize=(35, 5))

sns.boxplot(
    data=df_train, 
    x='additional_number_of_scoring', 
    ax=ax1[0]
);

sns.boxplot(
    data=df_train, 
    x='average_score', 
    ax=ax1[1]
);

sns.boxplot(
    data=df_train, 
    x='review_total_negative_word_counts', 
    ax=ax1[2]
);

sns.boxplot(
    data=df_train, 
    x='total_number_of_reviews', 
    ax=ax1[3]
);

sns.boxplot(
    data=df_train, 
    x='review_total_positive_word_counts', 
    ax=ax1[4]
);

sns.boxplot(
    data=df_train, 
    x='total_number_of_reviews_reviewer_has_given', 
    ax=ax1[5]
);

In [ ]:
display(data.describe(include=['object']))

Число уникальных значений в столбцах 'hotel_adress' и 'hotel_name'  совпадает

In [ ]:
# признаки непрерывного типа
print(data.select_dtypes(np.number).columns.to_list())

In [ ]:
# признаки категориального типа
print(data.select_dtypes('object').columns.to_list())

### 2.1 Дубликаты

In [ ]:
# Найдем дубликаты в датасете
dupl_columns = list(data.columns) 
mask = data.duplicated(subset=dupl_columns)
duplicates = data[mask]
print(f'Число найденных дубликатов: {duplicates.shape[0]}')

data_dedupped = data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data_dedupped.shape[0]}')

In [ ]:
# Удалим дубликаты из датасета
data.drop_duplicates(inplace=True, ignore_index=True)

### 2.2 Неинформативные признаки

In [ ]:
#список неинформативных признаков
low_information_cols = []

#цикл по всем столбцам
for col in data.columns:
    #наибольшая относительная частота в признаке
    top_freq = data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = data[col].nunique() / df_train[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100,2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
info_data = data_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {info_data.shape[1]}')

### 2.3 Пропуски

In [ ]:
cols_null_percent = data.isnull().mean()*100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
print(cols_with_null)

In [ ]:
null_data = data[data.isnull().values.any(axis=1)]
null_data['hotel_name'].value_counts()

Внесение координат на место пропусков

In [ ]:
# Список названий отелей с пропущенными координатами
names_list = null_data['hotel_name'].unique().tolist()
names_list

In [ ]:
# список координат для списка с названиями отеля
coordinates = [(48.220662,16.355555), (48.22092,16.366712), (48.888887,2.333167), (48.206487,16.36346), (48.21359,16.379881),
               (48.209477,16.35135), (40.874386,0.149246), (48.188578,16.382747), (48.87526,2.323403), (48.839813,2.323573),
               (48.2167,16.359983), (48.22005,16.284974), (41.712633,0.906265), (48.245935,16.341397), (48.186409,16.42017),
               (48.233495,16.345556), (41.379389,2.157475)]

In [ ]:
for name, coord in zip(names_list, coordinates):
    data.loc[data['hotel_name'] == name, ['lat', 'lng']] = coord[0], coord[1]
    
data[['lat', 'lng']].isnull().sum()

# 3. Извлечение информации из строковых данных и создание новых признаков


## 3.1 Работа с датами

In [ ]:
# Преобразуем дату в формат datetime
data['review_date'] = pd.to_datetime(data['review_date'])


# Посмотрим даты первого и последнего отзывов
data_min = data['review_date'].min()
data_max = data['review_date'].max()
print(f'Последний отзыв оставлен {data_max}. Первый отзыв оставлен {data_min}.')

# Извлечем из даты возраст отзыва, год, месяц и день недели
data['review_age'] = (pd.to_datetime('today') - data['review_date']).dt.days
data['review_day_of_week'] = data['review_date'].dt.weekday
data['review_month'] = data['review_date'].dt.month
data['review_year'] = data['review_date'].dt.year

# закодируем год
or_code = ce.OrdinalEncoder(cols=['review_year'])
data['year_code'] = or_code.fit_transform(data[['review_year']])

# Удалим признак "даты обзора" из данных
data.drop('review_date', axis=1 , inplace=True)

data.head(2)

## 3.2 Работа с текстом
### 3.2.1 Анализ адресов отеля

In [ ]:
# Смотрим уникальные адреса у отелей
data['hotel_address'].value_counts()

In [ ]:
#  Делаем функцию для создания признака "страна"
def get_country(address):
    words_list = address.strip().split(' ') # убираем пробелы в конце и в начале, разбиваем строку
    if 'United Kingdom' in address:
        return('United Kingdom')
    else:
        return(words_list[-1]) # используем индекс -1 так как в датасете страна написана в конце
    
# Используем созданную функцию на датасете
data['hotel_country'] = data['hotel_address'].apply(get_country)

# Подсчет уникальных значений
data['hotel_country'].value_counts()

In [ ]:
#  Делаем функцию для создания признака "город"
def get_city(address):
    words_list = address.split(' ') # разбиваем строку
    if 'United Kingdom' in address:
        return(words_list[-5]) # используем индекс -5 так как в United Kingdom используют индекс в адресе отеля
    else:
        return(words_list[-2]) # используем индекс -2 так как в датасете город написан перед страной

# Используем созданную функцию на датасете
data['hotel_city'] = data['hotel_address'].apply(get_city)

# Подсчет уникальных значений
data['hotel_city'].value_counts()

In [ ]:
# Проверяем все ли строки заполнены в признаке "страна"
data_country = data['hotel_country'].value_counts().sum() 
print(f'Количество строк в столбце "hotel_country" {data_country}.')

# Проверяем все ли строки заполнены в признаке "город"
data_city = data['hotel_city'].value_counts().sum()
print(f'Количество строк в столбце "hotel_city" {data_city}.')

Итог: наблюдаем равное количество строк в столбцах 'hotel_city' и 'hotel_country'. Соответственно можно удалить один из столбцов во избежания дублирования информации. 

In [ ]:
# Удаляем столбец с городами
data.drop('hotel_city', axis=1 , inplace=True)

# Удаляем столбец с адресом отелей
data.drop('hotel_address', axis=1 , inplace=True)

In [ ]:
# закодиируем столбец со странами
encoder = ce.OneHotEncoder(cols=['hotel_country'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['hotel_country'])
data = pd.concat([data, type_bin], axis=1)
data.info()

In [ ]:
# Преобразуем столбец с днями проверки отзывов
data['days_since_review'] = data['days_since_review'].apply(lambda x: x.split()[0]).astype('int')
data.info()

### 3.2.2 Анализ национальности рецензентов

In [ ]:
#Смотрим количество стран из которых были люди оставившие отзыв
nationality = data['reviewer_nationality'].nunique()
print(f'Количество стран из которых были люди оставившие отзыв: {nationality}.')

In [ ]:
# Создаем функцию, которая убирает пробелы в конце и в начале
def function_city(nation):
    nation = nation.strip()
    return nation 

# Применяем функцию удаляющую пробелы в конце и в начале
data['reviewer_nationality'] = data['reviewer_nationality'].apply(function_city)

# Создаем признак, отображающие есть ли люди путешествующие внутри своей страны
data['domestic_trip'] = data['reviewer_nationality'] == data['hotel_country']

# Проверяем наличие людей путешествующих внутри и за пределами своей страны
data['domestic_trip'].value_counts()

In [ ]:
# Кодируем названия отелей и национальность ревьюера посредсвом бинарного кодирования
bincode = ce.BinaryEncoder(cols=['hotel_name', 'reviewer_nationality'])
n_n_code = bincode.fit_transform(data[['hotel_name', 'reviewer_nationality']])
data = pd.concat([data, n_n_code], axis=1)

In [ ]:
data.info()

### 3.2.3 Анализ положительных и отрицательных отзывов

In [ ]:
# Создадим функцию, которая приведет отзывы к нижнему регистру и уберём пробелы в конце и в начале
def get_lower_strip(review):
    review = review.lower().strip()
    return review

# Применяем созданную функцию 
data['negative_review'] = data['negative_review'].apply(get_lower_strip)
data['positive_review'] = data['positive_review'].apply(get_lower_strip)

In [ ]:
# Посмотрим какие есть уникальные отрицательные отзывы
negative = data['negative_review'].value_counts()
print(negative[0:50])

# Посмотрим какие есть уникальные положительные отзывы
positive = data['positive_review'].value_counts()
print(positive[0:50])

#### 3.2.3.1 Признаки, связанные с номером

In [ ]:
# Создадим признак "проблема с номером"
data['room_bad'] = data['negative_review'].apply(lambda x: 1 if 'room' in x else 0) 
room_bad = data['room_bad'].value_counts() # проверяем работу lambda-функции
print(room_bad)

# Создадим признак "хорошая комната"
data['room_good'] = data['positive_review'].apply(lambda x: 1 if 'room' in x else 0)
room_good = data['room_good'].value_counts() # проверяем работу lambda-функции
print(room_good)


# Создадим признак "хорошая кровать"
data['bed_good'] = data['positive_review'].apply(lambda x: 1 if 'bed' in x else 0) 
bed_good = data['bed_good'].value_counts() # проверяем работу lambda-функции
print(bed_good)

# Создадим признак "проблема с кроватью"
data['bed_bad'] = data['negative_review'].apply(lambda x: 1 if ('bed' in x) or ('beds' in x) else 0) 
bed_bad = data['bed_bad'].value_counts() # проверяем работу lambda-функции
print(bed_bad)

#### 3.2.3.2 Признак - клининг комнаты

In [ ]:
data['cleaning_bad'] = data['negative_review'].apply(lambda x: 1 if ('cleaning' in x) or ('clean' in x) or ('dirt' in x) else 0) 
cleaning_bad = data['cleaning_bad'].value_counts() # проверяем работу lambda-функции
print(cleaning_bad)

data['cleaning_good'] = data['positive_review'].apply(lambda x: 1 if ('cleaning' in x) or ('clean' in x) or ('dirt' in x) else 0)  
cleaning_good = data['cleaning_good'].value_counts() # проверяем работу lambda-функции
print(cleaning_good)

#### 3.2.3.3 Признак - wi-fi

In [ ]:
# Создадим признак "проблема с wifi"
data['wifi_bad'] = data['negative_review'].apply(lambda x: 1 if ('wi-fi' in x) or ('internet' in x) or ('wi fi' in x) or ('wifi' in x) else 0) 
wifi_bad = data['wifi_bad'].value_counts() # проверяем работу lambda-функции
print(wifi_bad)

# Создадим признак "хорошая wi-fi"
data['wifi_good'] = data['positive_review'].apply(lambda x: 1 if ('wi-fi' in x) or ('internet' in x) or ('wi fi' in x) or ('wifi' in x) else 0) 
wifi_good = data['wifi_good'].value_counts() # проверяем работу lambda-функции
print(wifi_good)

#### 3.2.3.4 Признаки свзянные с персоналом

In [ ]:
# Создадим признак "проблема с работниками"
data['staff_bad'] = data['negative_review'].apply(lambda x: 1 if 'staff' in x else 0) 
staff_bad = data['staff_bad'].value_counts() # проверяем работу lambda-функции
print(staff_bad)

# Создадим признак "хороший персонал"
data['staff_good'] = data['positive_review'].apply(lambda x: 1 if 'staff' in x else 0)
staff_good = data['staff_good'].value_counts() # проверяем работу lambda-функции
print(staff_good)

#### 3.2.3.5 Признаки, связанные с качеством питания


In [ ]:
data['meal_service_bad'] = data['negative_review'].apply(lambda x: 1 if ('breakfast' in x) or ('eat' in x) or ('dinner' in x) or ('lunch' in x) or ('meal' in x) else 0)
meal_service_bad = data['meal_service_bad'].value_counts() # проверяем работу lambda-функции
print(meal_service_bad)

data['meal_service_good'] = data['positive_review'].apply(lambda x: 1 if ('breakfast' in x) or ('eat' in x) or ('dinner' in x) or ('lunch' in x) else 0)
meal_service_good = data['meal_service_good'].value_counts() # проверяем работу lambda-функции
print(meal_service_good)

#### 3.2.3.6 Признаки, связанные с местоположением отеля

In [ ]:
# Создадим признак 
data['location_bad'] = data['negative_review'].apply(lambda x: 1 if ('location' in x) or ('place' in x) else 0) 
location_bad = data['location_bad'].value_counts() # проверяем работу lambda-функции
print(location_bad)
# Создадим признак "хорошая локация"
data['location_good'] = data['positive_review'].apply(lambda x: 1 if ('location' in x) or ('place' in x) else 0) 
location_good = data['location_good'].value_counts() # проверяем работу lambda-функции
print(location_good)

#### 3.2.3.7 Признак - свежесть отеля 

In [ ]:
# Создадим признак "старое"
data['old'] = data['negative_review'].apply(lambda x: 1 if 'old' in x else 0) 
old = data['old'].value_counts() # проверяем работу lambda-функции
print(old)

# Создадим признак "новое"
data['new'] = data['positive_review'].apply(lambda x: 1 if 'new' in x else 0)  
new = data['new'].value_counts() # проверяем работу lambda-функции
print(new)

#### 3.2.3.8 Признаки, связанные с ценообразованием

In [ ]:
# Создадим признак "проблема с ценой"
data['price_bad'] = data['negative_review'].apply(lambda x: 1 if ('price' in x) or ('expensive' in x) or ('cost' in x) else 0)
price_bad = data['price_bad'].value_counts() # проверяем работу lambda-функции
print(price_bad)

# Создадим признак "хорошая цена"
data['price_good'] = data['positive_review'].apply(lambda x: 1 if ('price' in x) or ('cheap' in x) or ('cost' in x) else 0)
price_good = data['price_good'].value_counts() # проверяем работу lambda-функции
print(price_good)

#### 3.2.3.9 Признак - общее впечатление

In [ ]:
# Создадим признак "все не понравилось"
data['negative_all'] = data['negative_review'].apply(lambda x: 1 if ('all' in x) or ('everything' in x) else 0) 
negative_all = data['negative_all'].value_counts() # проверяем работу lambda-функции
print(negative_all)

# Создадим признак "все понравилось"
data['positive_all'] = data['positive_review'].apply(lambda x: 1 if ('all' in x) or ('everything' in x) else 0) 
positive_all = data['positive_all'].value_counts() # проверяем работу lambda-функции
print(positive_all)

In [ ]:
# Создаём списки с отсутствием позитивных и отсутствием негативных отзывов
no_negative_list = ['no negative', 'nothing', 'n a', 'none', ' ', 'nothing really', 
               'no complaints', 'nothing at all', 'nothing to dislike','nil', 
               'na', 'everything was perfect', 'everything was great', 
               'nothing to complain about', 'i liked everything',
               'liked everything', 'everything was good', 'nothing everything was great',
              'non', 'everything was fine']

no_positive_list = ['no positive', 'nothing','nothing ', ' ']


# Сигнальные признаки отсутсвия негативных и отсутвия позитивных отзывов
data['negative'] = data['negative_review'].apply(lambda x: 0 if x in no_negative_list else 1)
data['positive'] = data['positive_review'].apply(lambda x: 0 if x in no_positive_list else 1)

### 3.2.4 Признаки тэга

In [ ]:
# Создадим функцию, которая разбивает строку
def rev_tags(number_tags):
    number_tags = number_tags[2:-2] # указываем 2 и -2 чтобы исключить для дальнейшей работы скобки и пробел
    list_tags = number_tags.split(" ', '") # разбивает строку
    return list_tags

# Создадим отдельный признак со списками тегов
data['tags_list'] = data['tags'].apply(rev_tags)

# Создадим переменную, где каждый тег взят по отдельности
data_tag = data.explode('tags_list') 

# Смотрим количество уникальных тегов
data_tag['tags_list'].value_counts()[0:50]

#### 3.2.4.1 Признак - количество ночей

In [ ]:
# cоздадим признак количество ночей, если количество не указано, то поставим 0 ночей и преобразуем данные в числа
data['stayed_nights'] = data['tags'].str.extract(r'Stayed (\d+) night').fillna(0).astype(int)
data['stayed_nights'].value_counts() # проверяем результат

#### 3.2.4.2 Признак - кто остановился в отеле и по какой причине?

In [ ]:
# Создадим признак "бизнес путешествие"
data['business_trip'] = data['tags'].apply(lambda x: 1 if 'Business trip' in x else 0) 
print(data['business_trip'].value_counts()) # проверяем результат

# Создадим признак "путешествие на отдых"
data['leisure_trip'] = data['tags'].apply(lambda x: 1 if 'Leisure trip' in x else 0) 
print(data['leisure_trip'].value_counts()) # проверяем результат


# Создадим признак "путешествие с семьей"
data['traveling_with_family'] = data['tags'].apply(lambda x: 1 if 'Family' in x else 0) 
print(data['traveling_with_family'].value_counts()) # проверяем результат


# Создадим признак "путешествие в одиночку"
data['traveling_with_alone'] = data['tags'].apply(lambda x: 1 if 'Solo' in x else 0) 
print(data['traveling_with_alone'].value_counts()) # проверяем результат


# Создадим признак "путешествие в паре"
data['traveling_with_pairs'] = data['tags'].apply(lambda x: 1 if 'Couple' in x else 0) 
print(data['traveling_with_pairs'].value_counts()) # проверяем результат

#### 3.2.4.3 Признак - тип номера

In [ ]:
# Создадим признак "номер комфорт"
data['сomfort_room'] = data['tags'].apply(lambda x: 1 if 'Comfort' in x else 0) 
print(data['сomfort_room'].value_counts()) # проверяем результат

# Создадим признак "номер делюкс"
data['deluxe_room'] = data['tags'].apply(lambda x: 1 if 'Deluxe' in x else 0) 
print(data['deluxe_room'].value_counts()) # проверяем результат

# Создадим признак "номер класический"
data['сlassic_room'] = data['tags'].apply(lambda x: 1 if 'Classic' in x else 0) 
print(data['сlassic_room'].value_counts()) # проверяем результат

# Создадим признак "номер стандартный"
data['standard_room'] = data['tags'].apply(lambda x: 1 if 'Standard' in x else 0) 
print(data['standard_room'].value_counts()) # проверяем результат

# Создадим признак "номер люкс"
data['luxury_room'] = data['tags'].apply(lambda x: 1 if 'Luxury' in x else 0) 
print(data['luxury_room'].value_counts()) # проверяем результат

# Создадим признак "номер двойной"
data['double_room'] = data['tags'].apply(lambda x: 1 if 'Double' in x else 0) 
print(data['double_room'].value_counts()) # проверяем результат

#### 3.2.4.4 Признак - размещение с домашними животными

In [ ]:
data['with_a_pet'] = data['tags'].apply(lambda x: 1 if 'pet' in x else 0)
data['with_a_pet'].value_counts()

#### 3.2.4.5 Признак - способ размещения отзыва (через телефон или компьютер)

In [ ]:
data['mobile_device'] = data['tags'].apply(lambda x: 1 if  'mobile device' in x else 0)
data['mobile_device'].value_counts()

In [ ]:
data.info()

# 4 Нормализация данных

In [ ]:
# Создадим список с числовыми признаками (с выбросами), которые мы хотим нормализовать
cols = [
    'review_total_negative_word_counts', 'review_total_positive_word_counts', 
    'total_number_of_reviews', 'additional_number_of_scoring', 'total_number_of_reviews_reviewer_has_given'
]

In [ ]:
# зададим параметры холста, название и визуализируем кривые распределения:
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Исходные распределения')

# kdeplot() (KDE – оценка плотности ядра) – специальный метод для графиков распределений
sns.kdeplot(data['review_total_negative_word_counts'], ax = ax1, label='review_total_negative_word_counts')
sns.kdeplot(data['review_total_positive_word_counts'], ax=ax1, label='review_total_positive_word_counts')
sns.kdeplot(data['total_number_of_reviews'], ax=ax1, label='total_number_of_reviews')
sns.kdeplot(data['additional_number_of_scoring'], ax=ax1, label='additional_number_of_scoring')
sns.kdeplot(data['total_number_of_reviews_reviewer_has_given'], ax = ax1, label='total_number_of_reviews_reviewer_has_given')

plt.legend()

In [ ]:
from sklearn import preprocessing

# Проводим нормализацию RobustScaler, так как в датасете есть выбросы
# инициализируем нормализатор RobustScaler
r_scaler = preprocessing.RobustScaler()

col_names = list(data[cols])

# кодируем исходный датасет
data[cols] = r_scaler.fit_transform(data[cols])

# Преобразуем промежуточный датасет в полноценный датафрейм для визуализации
data_r = pd.DataFrame(data[cols], columns=col_names)

fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Распределения после RobustScaler')

sns.kdeplot(data_r['review_total_negative_word_counts'], ax=ax1, label='review_total_negative_word_counts')
sns.kdeplot(data_r['review_total_positive_word_counts'], ax=ax1, label='review_total_positive_word_counts')
sns.kdeplot(data_r['total_number_of_reviews'], ax=ax1, label='total_number_of_reviews')
sns.kdeplot(data_r['additional_number_of_scoring'], ax=ax1, label='additional_number_of_scoring')
sns.kdeplot(data_r['total_number_of_reviews_reviewer_has_given'], ax=ax1, label='total_number_of_reviews_reviewer_has_given')

plt.legend();

In [ ]:
# Колонны без выбросов
col_list = data[['days_since_review', 'review_age']].columns.tolist()

In [ ]:
# зададим параметры холста, название и визуализируем кривые распределения:
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Исходные распределения')

# kdeplot() (KDE – оценка плотности ядра) – специальный метод для графиков распределений
sns.kdeplot(data['days_since_review'], ax = ax1, label='days_since_review')
sns.kdeplot(data['review_age'], ax=ax1, label='review_age')

plt.legend()

In [ ]:
# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# кодируем исходный датасет
data[col_list] = mm_scaler.fit_transform(data[col_list])

# Смотрим на распределение
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 8))
ax1.set_title('Распределение после нормализации MinMaxScaler')
for col in col_list:
       sns.kdeplot(data[col], ax=ax1, label=col)
plt.legend();


In [ ]:
data.info()

# 5 Исследование данных

In [ ]:
# Строим график зависимости оценок пользователяк от месяца и года
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.lineplot(
    data = data,
    x = 'review_month',
    y = 'reviewer_score',
    hue = 'review_year',
    palette = 'tab10'
)
ax.set_title('Зависимость оценок пользователя от месяца и года', fontdict={'size': 14});


Из графика видно, что 
максимальную оценку ставят в феврале, после чего идет снижение, самые низкие оценки в мае-июне и сентябрь-октябрь.

Предполагаю, что это связано с общим количеством отзывов по месяцам. Так как летом люди чаще путешествуют, то и отзывов, в том числе и отрицательных, ставят больше.

In [ ]:
# Строим график зависимости количества оценок от месяца и года
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.histplot(
    data = data,
    x = 'review_month',
    hue = 'review_year',
    discrete=True,
    palette = 'tab10'
)
ax.set_title('Зависимость количества отзывов от месяца и года', fontdict={'size': 14});

Из графика видно, что больше всего отзывов оставляют в июль - август, это связано с тем, что чаще всего люди предпочитают брать отпуск и путешествовать летом, когда на улице хорошая погода. Меньше оставляют отзывов в ноябре, возможно это связано с плохой погодой и приближением конца года.

In [ ]:
# Строим график зависимости количества оценок от страны путешествия и от того гражданин путешествует или нет
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.histplot(
    data = data,
    x = 'hotel_country',
    hue = 'domestic_trip',
    palette = 'tab10',
    discrete=True
)
ax.set_title('Зависимость количества оценок от страны путешествия', fontdict={'size': 14});

# True - человек путешествует внутри своей страны
# False - человек приехал из другой страны

Из графика по нашем данным видно, что чаще всего путешествуют в United Kingdom, при чем в большей степени это внутренние путешествия. В других странах внутри страны очень мало путешествуют, в основном ездят за границу.

In [ ]:
# Строим график зависимости оценок пользователя от страны путешествия и месяца
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.lineplot(
    data = data,
    x = 'review_month',
    y = 'reviewer_score',
    hue = 'hotel_country',
    palette = 'tab10'
)
ax.set_title('Зависимость оценок пользователя от месяца и страны', fontdict={'size': 14});

На графике видно, что в целом в любой стране оценки распределена одинаково. За исключением Austria. Оценка за ее отели в феврале поднялась (может быть связана с открытиием новых отелей??). И United Kingdom имеет более низкие оценки за отели в сравнении с другими странами.


In [ ]:
# Строим график зависимости оценок пользователя от страны путешествия и от того гражданин путешествует или нет
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.lineplot(
    data = data,
    x = 'review_month',
    y = 'reviewer_score',
    hue = 'domestic_trip',
    palette = 'tab10'
)
ax.set_title('Зависимость оценок пользователя от месяца и внутреннего путешествия', fontdict={'size': 14});

# True - человек путешествует внутри своей страны
# False - человек приехал из другой страны

Из графика видно, что,  люди путешествующие внутри страны, ставят оценку выше, чем приехавшие.


In [ ]:
# Строим график зависимости количества оценок от количества ночей
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))

fig = sns.histplot(
    data = data,
    x = 'stayed_nights',
    discrete=True
)
ax.set_title('Зависимость количества оценок от количества ночей', fontdict={'size': 14});

Из графика мы видим, что больше всего оставляют отзывы люди останавливающиеся на 1 ночь. Чем больше ночей проводят люди в отеле, тем меньше оставляют отзывов. Предполагаю, что это связано с тем, что большинство людей останавливаются на несколько дней. Следовательно, отзывов они будут оставлять больше. Но почему значительно больше отзывов оставляют люди остановившиеся на 1 ночь, на данный момент сказать не могу. Нужны дальнейшее исследования данных.

In [ ]:
# Зависимость оценок пользователя от доли положительных и отрицательных отзывов
fig, (ax1) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

sns.lineplot(
    data=data, 
    x='positive_all', 
    y='reviewer_score', 
    ax=ax1[0])

sns.lineplot(
    data=data, 
    x='negative_all', 
    y='reviewer_score', 
    ax=ax1[1])

ax1[0].set_title('Зависимость оценки от положит. отзывов', fontdict={'fontsize': 14});
ax1[1].set_title('Зависимость оценки от отриц. отзывов', fontdict={'fontsize': 14});


Можем наблюдать на графиках, что отрицательные и положительные отзывы сильно влияют на оценку пользователя. Чем больше положительных отзывов, тем выше оценка, и чем больше отрицательных, тем ниже будет оценка.

#### Гипотеза

Нулевая гипотеза: нет значимой разницы между средней оценкой, которую рецензент поставил отелю и средним баллом отеля. Альтернативная гипотеза: есть значимая разница между средней оценкой, которую рецензент поставил отелю и средним баллом отеля.

Для проверки гипотезы применю тест ANOVA

In [ ]:
from scipy.stats import f_oneway

reviewer = data['reviewer_score']
average = data['average_score']

alpha = 0.05
_, p = f_oneway(reviewer, average)

H0 = 'Нет значимой разницы между средней оценкой, которую рецензент поставил отелю и  средним баллом отеля'
H1 = 'Есть значимая разница между средней оценкой, которую рецензент поставил отелю и  средним баллом отеля.'

if p>alpha:
  print(f'{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}')
else:
  print(f'{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {H1}')

# 6 Анализ и отбор признаков

In [ ]:
data_copy = data.copy()

In [ ]:
print(f'Текущее получившееся количество признаков: {len(data_copy.columns)}')
print('(включая целевой столбец и метку тест/трейн)')


# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']

print(f'Удалим {len(object_columns)} столбцов с нечисловыми данными')
data_copy.drop(object_columns, axis = 1, inplace=True)

In [ ]:
# Заменим булевые значения в признаке domestic_trip на числа 0 и 1
data_copy['domestic_trip'] = data_copy['domestic_trip'].replace(True,1)
data_copy['domestic_trip'] = data_copy['domestic_trip'].replace(False,0)

In [ ]:
# data_copy.columns.tolist()

## Анализ мультиколлинеарности

In [ ]:
# Построим корреляцию и отфильтруем по очень сильной взаимосвязи
df_corr = data_copy.corr()
df_corr = df_corr[(df_corr < 1) & (abs(df_corr) > 0.7)]
df_corr = df_corr.dropna(thresh=1, axis=0)
df_corr = df_corr.dropna(thresh=1, axis=1)
matrix = np.triu(df_corr)

# Визуализируем матрицу корреляции
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.heatmap(df_corr, annot=True, ax=ax, mask=matrix, linewidths=1, center=0, vmin=-1);

In [ ]:
# Удалим мультиколлинеальные признаки
data_copy = data_copy.drop([
    'additional_number_of_scoring', 'year_code', 'review_year', 'leisure_trip',
    'hotel_country_Spain', 'hotel_country_Austria', 'review_age'
], axis=1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
mask = np.array(abs(data_copy.corr()) < 0.7)
sns.heatmap(data_copy.corr(), annot=True, ax=ax, center=0, vmin=-1, linewidths=1, mask=mask);

In [ ]:
data_copy.info()

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data_copy.columns if data_copy[s].dtypes == 'object']
data_copy.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data_copy.info()

## Отбор признаков

In [ ]:
# Теперь выделим тестовую часть
train_data = data_copy.query('sample == 1').drop(['sample'], axis=1)
test_data = data_copy.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
from sklearn.feature_selection import chi2 

# категориальные признаки
cat_cols = [
    'average_score', 'hotel_country_Italy', 'hotel_country_Netherlands',
    'hotel_country_United Kingdom', 'hotel_country_France', 
    'domestic_trip', 'hotel_name_0', 'hotel_name_1', 'hotel_name_2',
    'hotel_name_3', 'hotel_name_4', 'hotel_name_5', 'hotel_name_6',
    'hotel_name_7', 'hotel_name_8', 'hotel_name_9', 'hotel_name_10',
    'reviewer_nationality_0', 'reviewer_nationality_1', 'reviewer_nationality_2',
    'reviewer_nationality_3', 'reviewer_nationality_4', 'reviewer_nationality_5',
    'reviewer_nationality_6', 'reviewer_nationality_7', 'room_bad', 'room_good',
    'bed_good', 'bed_bad', 'cleaning_bad', 'cleaning_good', 'wifi_bad', 'wifi_good',
    'staff_bad', 'staff_good', 'meal_service_bad', 'meal_service_good', 
    'location_bad', 'location_good', 'old', 'new', 'price_bad', 'price_good',
    'negative_all','positive_all', 'negative', 'positive', 'stayed_nights', 
    'business_trip', 'traveling_with_family', 'traveling_with_alone',
    'traveling_with_pairs', 'сomfort_room', 'deluxe_room', 'сlassic_room',
    'standard_room', 'luxury_room', 'double_room', 'with_a_pet', 'mobile_device'
]

y = y.astype('int')
# Проанализируем важность категориальных признаков
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh');

In [ ]:
# Уберем малозначимые признаки
data_copy.drop([
    'hotel_name_0','hotel_name_1', 'hotel_name_2','hotel_name_3','hotel_name_4', 'hotel_name_5',
    'hotel_name_6','hotel_name_7','hotel_name_8','hotel_name_9','hotel_name_10',
    'reviewer_nationality_0','reviewer_nationality_1','reviewer_nationality_2', 
    'reviewer_nationality_4','reviewer_nationality_6', 'reviewer_nationality_7',
    'reviewer_nationality_5', 'with_a_pet', 'double_room','сomfort_room','luxury_room',
    'deluxe_room',    'сlassic_room','hotel_country_Italy', 'hotel_country_Netherlands',
    'hotel_country_France', 'hotel_country_United Kingdom','traveling_with_family',
    'domestic_trip', 'wifi_good','price_good','stayed_nights','new'
], axis=1, inplace=True)

In [ ]:
from sklearn.feature_selection import chi2 

# категориальные признаки
cat_cols = [
    'average_score', 'room_bad', 'room_good',
    'bed_good', 'bed_bad', 'cleaning_bad', 'cleaning_good', 'wifi_bad', 
    'staff_bad', 'staff_good', 'meal_service_bad', 'meal_service_good', 
    'location_bad', 'location_good', 'old', 'price_bad', 
    'negative_all','positive_all', 'negative', 'positive',  
    'business_trip',  'traveling_with_alone',
    'traveling_with_pairs', 'standard_room', 'mobile_device'
]

y = y.astype('int')
# Проанализируем важность категориальных признаков
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh');

In [ ]:
from sklearn.feature_selection import f_classif # anova

# Непрерывные признаки
num_cols = [
    'total_number_of_reviews', 'review_total_negative_word_counts', 'review_total_positive_word_counts',
    'total_number_of_reviews_reviewer_has_given', 'average_score', 'lat', 'lng'
]
# average_score проверила в категориальных признаках, так было написано в примере в юнете, а так же
# решила проверить в непрерывных, так как данное число должно постепенно меняться 

# Проанализируем важность непрерывных признаков
imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh');

In [ ]:
# Уберем малозначимые признаки
data_copy.drop(['total_number_of_reviews', 'total_number_of_reviews_reviewer_has_given', 'lat', 'lng'
          ], axis=1, inplace=True)

In [ ]:
data = data_copy

# 7. Построение модели 

In [ ]:
plt.rcParams['figure.figsize'] = (35,30)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission


In [ ]:
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)